In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from laspy.file import File
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

import sys
sys.path.insert(0,'..')
from point_density_functions import *

%matplotlib notebook

In [2]:
file_dir = '../../Data/parking_lot/'

filenames =list(pd.read_csv(file_dir+"filenames.txt",header=None)[0])
pt_files = list(pd.read_csv(file_dir+"pt_files.txt",header=None)[0])

nyc_file_dir = '../../Data/NYC_topo/'
nyc_pt_file = ['las_points_NYC_975172.lz']

# Corresponds to LAS 1.2 Point Data Record Format 1
columns_dublin_pt_cloud = [
    'X',
    'Y',
    'Z',
    'intensity',
    'return_number_byte',
    'classification_byte',
    'scan_angle',
    'user_data',
    'pt_src_id',
    'gps_time']

columns_point_cloud = [
    'X','Y','Z',
    'intensity',
    'flag_byte',
    'classification_flags',
    'classification_byte',
    'user_data',
    'scan_angle',
    'pt_src_id',
    'gps_time']

In [3]:
# Extract points within a square around the desired point

# Parking Lot
# pt_x = 977037.343
# pt_y = 174586.034

# Corner of building
# pt_x_bldg = 977229.375
# pt_y_bldg = 174579.42

# Top of building
# pt_x = 977229.375
# pt_y = 174579.42

# Projects in back parking lot
# pt_x = 977458.238
# pt_y = 173302.388

# Solar panel
# pt_x = 977682.975
# pt_y = 174148.192

In [4]:
# This works
# for filename in filenames:
#     create_df_hd5(file_dir,filename,columns_point_cloud)

## Flat surface
Identifying points in a parking lot to assess how consistently flat they are.  
Center point: 40.645789, -74.025951  
Easting - 977048.434  
Northing - 174555.792

### Point density - waterside parking lot

In [5]:
# Big parking lot rectangle
def rectangle(pt1,pt2,y_length,x_length):
    '''
    Inputs:
    pt1 - 2x1 numpy array with x and y coordinate for bottom point
    pt2 - 2xy numpy array with x and y coordinate for top point
    y_length - bottom-to-top length (positive is in direction of top from bottom point)
    x_length - left-to-right length (positive means pts are on left border, negative means they're on right)
    '''
    unit_u = (pt2 - pt1)/np.linalg.norm(pt2-pt1)
    unit_v = np.array([unit_u[1],-1*unit_u[0]])
    u = unit_u*y_length
    v = unit_v*x_length
    uv = np.array([u,v]).T
    uv_inv = np.linalg.inv(uv)
    w = pt1
    return uv_inv,w

pt1 = np.array([976600.43, 173886.11])
pt2 = np.array([976774.81, 174083.03])
uv_inv,w = rectangle(pt1,pt2,800,-80)


rectangle_points_laefer = grab_points_big_rect(pt_files,file_dir,uv_inv,w)
rectangle_points_nyc = grab_points_big_rect(nyc_pt_file,nyc_file_dir,uv_inv,w)

Point count in new square from las_points_161348.lz: 0
Point count in new square from las_points_161518.lz: 0
Point count in new square from las_points_161726.lz: 0
Point count in new square from las_points_161905.lz: 0
Point count in new square from las_points_162124.lz: 0
Point count in new square from las_points_162306.lz: 0
Point count in new square from las_points_162537.lz: 0
Point count in new square from las_points_162730.lz: 0
Point count in new square from las_points_162957.lz: 0
Point count in new square from las_points_163206.lz: 0
Point count in new square from las_points_163425.lz: 0
Point count in new square from las_points_163824.lz: 0
Point count in new square from las_points_164039.lz: 81892
Point count in new square from las_points_164239.lz: 134380
Point count in new square from las_points_164445.lz: 193394
Point count in new square from las_points_164640.lz: 143754
Point count in new square from las_points_164845.lz: 154601
Point count in new square from las_points

In [6]:
# Remove outliers and print the count: z < 0, z > 30
def remove_vertical_outliers(rectangle_points,z_low,z_high):
    outliers = rectangle_points[(rectangle_points['z_scaled']<z_low) | (rectangle_points['z_scaled']>z_high)].index
    rectangle_points = rectangle_points.drop(outliers)
    print("Number of outliers: {}".format(len(outliers)))
    return rectangle_points
rectangle_points_nyc = remove_vertical_outliers(rectangle_points_nyc,0,30)
rectangle_points_laefer = remove_vertical_outliers(rectangle_points_laefer,0,30)

Number of outliers: 389
Number of outliers: 1004


In [28]:
def random_rectangle_points(num_points,pt1,pt2,y_length,x_length,border=0.05):
    '''
    Inputs:
    pt1 - 2x1 numpy array with x and y coordinate for bottom point
    pt2 - 2xy numpy array with x and y coordinate for top point
    y_length - bottom-to-top length (positive is in direction of top from bottom point)
    x_length - left-to-right length (positive means pts are on left border, negative means they're on right)
    border - portion of unit square on each edge to avoid
    
    Output:
    num_points x 2 numpy array of (x,y) values
    '''
    # Calculate uv and w for use in translation
    unit_u = (pt2 - pt1)/np.linalg.norm(pt2-pt1)
    unit_v = np.array([unit_u[1],-1*unit_u[0]])
    u = unit_u*y_length
    v = unit_v*x_length
    uv = np.array([u,v]).T
    w = pt1
    
    # Select random point on unit square within border
    square_side = 1 - (2*border)
    st = border + (square_side)*np.random.rand(2,num_points)
    return (uv @ st + w.reshape((2,1))).T

def in_square(rectangle_points,center_point,feet_from_point):
    '''
    Filters dataframe for points less than feet_from_point from center_point
    Inputs:
    rectangle_points - (n x 3+) dataframe with fields x_scaled, y_scaled, and z_scaled
    center_point - 2x1 (x,y) point within rectangle
    feet_from_point - scalar
    
    Output:
    square_points - (n x 3+) filtered dataframe
    '''
    square_points = rectangle_points[ (rectangle_points['x_scaled'] < center_point[0] + feet_from_point)
            &(rectangle_points['x_scaled'] > center_point[0] - feet_from_point) 
            &(rectangle_points['y_scaled'] < center_point[1] + feet_from_point)
            &(rectangle_points['y_scaled'] > center_point[1] - feet_from_point)
          ]
    return square_points


feet_from_point = 2
center_points = random_rectangle_points(1000,pt1,pt2,800,-80,border=0.05)

# Aggregators
pt_density_nyc_list, pt_density_laefer_list = [],[]
avg_height_diff = []
sd_height_nyc, sd_height_laefer = [],[]
pts_thrown_out = 0
for center_point in center_points:
    square_points_nyc = in_square(rectangle_points_nyc,center_point,feet_from_point)
    square_points_laefer = in_square(rectangle_points_laefer,center_point,feet_from_point)
    
    # If z_max > 10, <6 for either dataset, skip it
    if (square_points_nyc['z_scaled'].max()<10) & (square_points_nyc['z_scaled'].min()>6) & \
       (square_points_laefer['z_scaled'].max()<10) & (square_points_laefer['z_scaled'].min()>6):
        # Statistics!
        
        # Point density
        num_points_nyc = square_points_nyc.shape[0]
        pt_density_nyc_list.append(num_points_nyc / (4 * feet_from_point))        
        num_points_laefer = square_points_laefer.shape[0]
        pt_density_laefer_list.append(num_points_laefer / (4 * feet_from_point))        

        # Average height, SD from height
        avg_height_nyc = square_points_nyc['z_scaled'].mean()
        avg_height_laefer = square_points_laefer['z_scaled'].mean()
        avg_height_diff.append(avg_height_nyc-avg_height_laefer)
        sd_height_nyc.append(np.std(square_points_nyc['z_scaled']))
        sd_height_laefer.append(np.std(square_points_laefer['z_scaled']))
    else:
        pts_thrown_out +=1

In [34]:
print("NYC avg density: {:2.4f} (SD: {:2.4f})".format(np.mean(pt_density_nyc_list),np.std(pt_density_nyc_list)))
print("Laefer avg density: {:2.4f} (SD: {:2.4f})".format(np.mean(pt_density_laefer_list),np.std(pt_density_laefer_list)))
print("Points thrown out: {} ({:2.2%})".format(pts_thrown_out,pts_thrown_out/len(center_points)))
print("\n")
print("Mean and Std of avg height difference: {:2.2f} (SD: {:2.2f})".format(np.mean(avg_height_diff),np.std(avg_height_diff)))

NYC avg density: 2.3196 (SD: 0.6697)
Laefer avg density: 93.3051 (SD: 7.7874)
Points thrown out: 258 (25.80%)


Mean and Std of avg height difference: 0.12 (SD: 0.11)


In [35]:
rectangle_points_laefer.to_pickle("../../Data/parking_lot/rectangle_points_laefer.pkl")
rectangle_points_nyc.to_pickle("../../Data/parking_lot/rectangle_points_nyc.pkl")

In [ ]:
# Parking Lot
pt_x = 977037.343
pt_y = 174586.034
feet_from_pt = 5
# square_points_laefer = grab_points(pt_files,file_dir,pt_x,pt_y,feet_from_pt)
# Remove 3-point flight 200024
# square_points_laefer = square_points_laefer[square_points_laefer['flight_id']!='200024']
square_points_nyc = grab_points(nyc_pt_file,nyc_file_dir,pt_x,pt_y,feet_from_pt)

In [ ]:
# Fit a plane to each flight in laefer and nyc, and one overall plane
total_plane_norm,total_points = plane_fit(square_points_laefer)[:2]
nyc_plane_norm,nyc_points = plane_fit(square_points_nyc)[:2]
flight_ids = square_points_laefer['flight_id'].unique()
flight_plane_norms_dict ={}
for flight_id in flight_ids:
    flight_plane_norms_dict[flight_id] = plane_fit(square_points_laefer[square_points_laefer['flight_id']==flight_id])[:2]

In [ ]:
# Compare the normal vectors...cosine similarity?
# Dataframe of the cosine similarities
cosine_simarlity_df = pd.DataFrame(index = flight_ids,columns = flight_ids,dtype=float)
for flight_id_ix in flight_ids:
    for flight_id_col in flight_ids:
        cosine_simarlity_df.loc[flight_id_ix,flight_id_col] = float(flight_plane_norms_dict[flight_id_ix][0]@ \
                                                              flight_plane_norms_dict[flight_id_col][0])


In [ ]:
sns.heatmap(cosine_simarlity_df)

In [ ]:
# Calculate avg pt distance from flight plane and overall plane for each flight

square_points_laefer['dist_from_total_plane'] = np.array([np.dot(point,np.array([0,0,1])) for point in total_points])
square_points_nyc['dist_from_total_plane'] = np.array([np.dot(point,np.array([0,0,1])) for point in nyc_points])
# Compare whole process to splitting the points randomly instead of by flight

In [ ]:
print("Mean point distance from plane (laefer): {:2.4f}".format(abs(square_points_laefer['dist_from_total_plane']).mean()))
print("Mean point distance from plane (nyc): {:2.4f}".format(abs(square_points_nyc['dist_from_total_plane']).mean()))

## Vertical density
Identifying point at corner of building to quantify the vertical point density.  
Center point: 	40.645854, 	-74.025299  
Easting - 977229.375  
Northing - 174579.42

#### Laefer data

In [ ]:
# Corner of building
pt_x_bldg = 977222.375
pt_y_bldg = 174577.42
feet_from_pt = 20

square_points_bldg = grab_points(pt_files,file_dir,pt_x_bldg,pt_y_bldg,feet_from_pt)

# Remove one bad point if present in dataset
if square_points_bldg['z_scaled'].min() < -50:
    bad_point_ix = square_points_bldg['z_scaled'].argmin()
    square_points_bldg = square_points_bldg.drop(bad_point_ix,axis=0)
square_points_bldg,min_list = prep_square_for_plotting(square_points_bldg)

#### NYC data

In [ ]:
# Access the NYC header file
#inFile = File(nyc_file_dir+'975172.las', mode='r')

# Extract points within a square around the desired point
# Use pt_[]_bldg and feet_from_pt from previous cell

nyc_file_dir = '../../Data/NYC_topo/'
nyc_pt_file = ['las_points_NYC_975172.lz']
square_points_nyc = grab_points(nyc_pt_file,nyc_file_dir,pt_x_bldg,pt_y_bldg,feet_from_pt)
# Use same min's from previous so points line up.
square_points_nyc,_ = prep_square_for_plotting(square_points_nyc,min_list) 

#### 3D Plots

In [ ]:
fig = px.scatter_3d(square_points_bldg, x='x_plot', y='y_plot', z='z_plot',
              size='size_num',size_max = 12)

fig.update_layout( 
    scene = dict(xaxis = dict(title="Easting (feet)"),
                 yaxis = dict(title="Northing (feet)"),
                 zaxis = dict(title="Vertical (feet)"),
                ),
    width=900,
    height=900,
    margin=dict(r=20, l=10, b=10, t=10),
    showlegend=False,
    )
fig.show()

In [ ]:
fig = px.scatter_3d(square_points_nyc, x='x_plot', y='y_plot', z='z_plot',
              size='size_num',size_max = 12)

fig.update_layout( 
    scene = dict(xaxis = dict(title="Easting (feet)"),
                 yaxis = dict(title="Northing (feet)"),
                 zaxis = dict(title="Vertical (feet)"),
                ),
    width=900,
    height=900,
    margin=dict(r=20, l=10, b=10, t=10),
    showlegend=False,
    )
fig.show()

#### Extract the wall face, find normal vector, calculate vertical density

In [ ]:
pt_1 = np.array([37.7,10.04])
pt_2 = np.array([27.094,18.439])
wall_pt = (31.77,14.90,43.00)

# East? Facing Wall (farther from the little stump)
# pt_1 = np.array([20.59,15.02])
# pt_2 = np.array([10.34,2.93])
# wall_pt = (19.414,12.379,73.863)

wall_face = grab_wall_face(square_points_bldg,pt_1,pt_2,20,100,1e-2)
wall_face_nyc = grab_wall_face(square_points_nyc,pt_1,pt_2,20,100,5e-2)

In [ ]:
wall_face.shape

In [ ]:
wall_face_nyc.shape

In [ ]:
norm_vector,points,wall_face,pts_on_plane = plane_fit(wall_face)

In [ ]:
print(norm_vector)
print(norm_vector_nyc)

In [ ]:
norm_vector_nyc,points,wall_face_nyc,pts_on_plane_nyc = plane_fit(wall_face)

In [ ]:
fig = px.scatter_3d(wall_face, x='x_plot', y='y_plot', z='z_plot',
              color='flight_id',size='size_num',size_max = 12)

fig.update_layout( 
    scene = dict(xaxis = dict(title="Easting (feet)"),
                 yaxis = dict(title="Northing (feet)"),
                 zaxis = dict(title="Vertical (feet)"),
                ),
    width=900,
    height=900,
    margin=dict(r=20, l=10, b=10, t=10),
    showlegend=False,
    )
fig.show()

In [ ]:
# Point in middle of wall - in xyz_plot coordinates (less the min of each coordinate)
# It's best to highlight a point in the plot above and use that.

feet_from_pt = 3

In [ ]:
# Calculate density for both datasets   
vertical_point_density(square_points_nyc,norm_vector_nyc,wall_pt,feet_from_pt)
vertical_point_density(square_points_bldg,norm_vector,wall_pt,feet_from_pt)

In [ ]:
fig = px.scatter_3d(vertical_square, x='x_plot', y='y_plot', z='z_plot',
              color='flight_id', size='size_num',size_max = 8)

fig.update_layout( 
    scene = dict(xaxis = dict(title="Easting (feet)"),
                 yaxis = dict(title="Northing (feet)"),
                 zaxis = dict(title="Vertical (feet)"),
                ),
    width=900,
    height=900,
    margin=dict(r=20, l=10, b=10, t=10),
    showlegend=False,
    xaxis = {"title":{"text":"Cat"}})
fig.show()

### Writing LAS file

In [ ]:
# Write square_points_bldg to file
inFile = File(file_dir+'10552_NYU_M2 - Scanner 1 - 190511_164039_1 - originalpoints.laz', mode='r')
# Convert DF into tuples that laspy wants
void = [(tuple(r[columns_point_cloud]),) for i,r in square_points_laefer.iterrows()]
# Export
outFile1 = File("../../Data/parking_lot/flat_parking_lot_laefer.las", mode = "w",header = inFile.header)
outFile1.points = void
outFile1.close()

In [ ]:
# NYC data
# Write square_points_bldg to file
inFile_nyc = File(nyc_file_dir+'975172.las', mode='r')
# Convert DF into tuples that laspy wants
void = [(tuple(r[columns_point_cloud]),) for i,r in square_points_nyc.iterrows()]
# Export
outFile1 = File("../../Data/parking_lot/flat_parking_lot_nyc.las", mode = "w",header = inFile.header)
outFile1.points = void
outFile1.close()

## Plotting charts from previous updates

In [ ]:
norm_vector,points,square_points,_ = plane_fit(square_points)

# Add distance from flat plane with norm (x,y,z) = (0,0,1)
square_points['dist_from_flat']=np.array([np.dot(point,np.array([0,0,1])) for point in points])

# remove data points >5 feet below plane.
outliers = square_points[square_points['dist_from_plane']<-5].index
square_points = square_points.drop(outliers)

In [ ]:
def plot_scan_angle_dist_from_plane(df,distance_metric):
    x = abs(df['scan_angle'])*.006
    y = df[distance_metric]
    plt.figure(figsize=(15,15))
    plt.plot(x,y,'xb')
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    plt.plot(x,p(x),"r--")
    plt.xlabel("Scan angle (degrees)")
    plt.ylabel("Point distance from plane")
    print("y={:2.8f}x+{:2.8f}".format(z[0],z[1]))
    plt.title("Scan Angle vs Distance to Fitted Plane")
plot_scan_angle_dist_from_plane(square_points,'dist_from_plane')

In [ ]:
plt.plot(range(len(square_points)),square_points['scan_angle'],'x')

In [ ]:
plot_scan_angle_dist_from_plane(square_points,'dist_from_flat')

In [ ]:
# Chart from slides showing points per run
labels = [pt[0][11:-4] for pt in pts_from_scan]
num_points = [pt[1]+.01 for pt in pts_from_scan]
plt.figure(figsize=(25,20))
plt.bar(labels,num_points,)
plt.xticks(rotation=45,fontsize=20)
plt.yticks(np.arange(0, max(num_points), step=(max(num_points)/10)),fontsize=20)
plt.ylabel("Number of points from run",fontsize=20)
plt.xlabel("Run ID",fontsize=20)